In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
with open('../data/puzzles.json', 'r') as jsonfile:
    puzzle_data = json.load(jsonfile)

In [3]:
puzzles = []
solutions = []
for puzzle in puzzle_data:
    puzzles.append(puzzle["puzzle"])
    solutions.append(puzzle["solution"])
puzzles = np.array(puzzles)
solutions = np.array(solutions)

In [4]:
len(puzzles)

5300

In [10]:
puzzles_train, puzzles_test, solutions_train, solutions_test = train_test_split(puzzles, solutions, test_size=0.2, random_state=42)

In [11]:
len(puzzles_train)

4240

In [12]:
puzzles_train[0]

array([[5, 0, 0, 6, 7, 3, 1, 0, 2],
       [0, 0, 7, 0, 9, 4, 0, 3, 6],
       [0, 0, 4, 2, 5, 1, 8, 0, 9],
       [0, 9, 0, 4, 0, 8, 7, 0, 1],
       [2, 7, 6, 0, 0, 0, 9, 8, 4],
       [0, 0, 1, 0, 2, 7, 3, 0, 0],
       [0, 6, 2, 1, 8, 9, 0, 5, 0],
       [0, 0, 3, 0, 0, 6, 2, 1, 8],
       [4, 1, 0, 5, 3, 0, 0, 9, 7]])

In [ ]:
# MSE not appropriate --> will lead to 4 and 5 guesses
# Should be either correct or wrong
# Count the number of wrong ones as loss function
# WTF is sparse_categorical_crossentropy

In [13]:
puzzles_train = puzzles_train.reshape(-1, 81) / 9
puzzles_test = puzzles_test.reshape(-1, 81) / 9
solutions_train = solutions_train.reshape(-1, 81) / 9
solutions_test = solutions_test.reshape(-1, 81) / 9

In [18]:
def create_sudoku_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(81,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(81, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = create_sudoku_model()

In [22]:
new_puzzle = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

# Reshape the input to match the model's input shape
new_puzzle = new_puzzle.flatten()
new_puzzle = new_puzzle / 9.0  # Normalize the values between 0 and 1

# Perform the prediction
predictions = model.predict(np.array([new_puzzle]))

# Reshape the predictions back to the 9x9 grid shape
predictions = predictions.reshape((9, 9))

# Find the most probable values for each cell
solved_puzzle = np.argmax(predictions, axis=1)

# Reshape the solved puzzle back to the 9x9 grid shape
solved_puzzle = solved_puzzle.reshape((9, 9))

# Print the solved puzzle
print(solved_puzzle)

1/1 [==============================] - 0s 20ms/step


ValueError: cannot reshape array of size 9 into shape (9,9)

In [21]:
predictions

array([[0.01437321, 0.01560794, 0.01248061, 0.01009885, 0.01462195,
        0.01161344, 0.0138914 , 0.00972902, 0.01197504, 0.00755326,
        0.00969903, 0.01370753, 0.01155839, 0.01203171, 0.00870537,
        0.01026498, 0.01112598, 0.01366768, 0.01371307, 0.0101701 ,
        0.00903686, 0.01292316, 0.01086591, 0.01606416, 0.01072702,
        0.01021174, 0.01686424, 0.01259067, 0.01096443, 0.01021778,
        0.01727676, 0.01296812, 0.01361099, 0.01072106, 0.01223464,
        0.01124933, 0.01294846, 0.01115233, 0.00889868, 0.01216738,
        0.01219465, 0.01114055, 0.01569491, 0.01069487, 0.01151508,
        0.01095222, 0.00811825, 0.01361703, 0.01232372, 0.01152715,
        0.01208541, 0.01067103, 0.01774103, 0.01340362, 0.01457326,
        0.01028982, 0.01236868, 0.01076449, 0.00949747, 0.01465834,
        0.01613144, 0.01293257, 0.01011906, 0.01301018, 0.01873202,
        0.01287124, 0.01363545, 0.01536106, 0.01407051, 0.01340923,
        0.01485399, 0.00880601, 0.00782947, 0.01

In [14]:
puzzles_test[0]

array([0.55555556, 0.        , 0.        , 0.11111111, 0.        ,
       0.        , 0.88888889, 0.77777778, 0.44444444, 0.77777778,
       0.33333333, 0.        , 0.22222222, 0.66666667, 0.44444444,
       1.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.77777778, 0.88888889, 0.        , 0.        ,
       0.33333333, 0.66666667, 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.66666667, 0.        , 0.44444444,
       0.33333333, 0.22222222, 0.66666667, 0.        , 0.        ,
       0.11111111, 1.        , 0.77777778, 0.55555556, 0.88888889,
       0.44444444, 0.11111111, 0.55555556, 0.33333333, 0.        ,
       0.        , 0.        , 0.22222222, 0.        , 0.        ,
       0.        , 0.22222222, 0.88888889, 0.55555556, 0.11111111,
       0.        , 0.66666667, 0.        , 0.66666667, 0.        ,
       0.        , 0.        , 0.33333333, 0.22222222, 0.44444444,
       0.        , 0.        , 0.        , 0.88888889, 0.11111

In [28]:
model.fit(puzzles_train, solutions_train, epochs=10, batch_size=32)

loss, accuracy = model.evaluate(puzzles_test, solutions_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\woute\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\woute\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "c:\Users\woute\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 724, in start
      self.io_loop.start()
    File "c:\Users\woute\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\woute\AppData\Loca

In [ ]:
model.save('solve_model.h5')